<img src="https://raw.githubusercontent.com/fluidml/fluidml/main/logo/fluid_ml_logo.png" width="400px"/>

# **Text Classification using FluidML and Sklearn**
In this notebook, we'll go over some basics of FluidML and implement a complete ML pipeline that performs text classification.
Like any other ML pipeline, this usually consists of several tasks:
- **Dataset fetching** - Downloads and parses the dataset for text classification
- **Dataset pre-processing** - Pre-processes the raw dataset
- **Featurization** - Converts the raw sentences to numerical vectors
- **Training a classifier** - Trains a logistic regression model
- **Evaluation of classifier** - Evaluates the trained model on train/test splits

With FluidML, all of these steps are naturally implemented as individual tasks which register their dependencies and are chained together to a task graph. This graph is then executed in parallel by FluidML and all results are returned at the end.

## **Setup**

To run this example, make sure to install FluidML with the additional example requirements.

In [ ]:
!pip install fluidml[examples]

**Note 1**: Due to the limitation of multiprocessing and jupyter, we have to import our defined tasks and some helper classes from a separate script. Hence, our task definitions are located in `sklearn_text_classification.py`, which not only implements the tasks but also the entire functionality of this example. So the interested reader can also go ahead and execute the just mentioned script. In order to still make this notebook self-explanatory, we provide Markdown code snippets of the individual task implementations at the place where we would have defined the task.

In [1]:
from fluidml import Task, Flow, TaskSpec

from sklearn_text_classification import DatasetFetchTask, PreProcessTask, TFIDFFeaturizeTask, GloveFeaturizeTask, TrainTask, EvaluateTask, ModelSelectionTask
from rich import print

**Note 2**: If you want to use FluidML's logging capability, please configure a logger using Python's `logging` API. For convenience, we provide a simple utility function which configures a visually appealing logger (using a specific handler from the `rich` library).

In [2]:
from fluidml.logging import configure_logging
configure_logging(level='INFO')

## **Task Definitions**

### **1. Dataset Fetching**

Let's use HuggingFace's [datasets](https://huggingface.co/datasets) repository to quick get access to a text classification dataset. Specifically, we will use [TREC](https://huggingface.co/datasets/trec) which is a question classification dataset containing ~ 5k labeled questions in training set and ~500 questions in test set. The dataset contains two types of labels: fine and coarse. For simplicity, let's go ahead with coarse, with unique 6 labels.

We can implement this dataset collection as a separate task on its own by inheriting from FluidML's Task class. It just has to implement a run() method and publish its results. 
For simplicity, we can implement this task to publish a nested dictionary. On the outer level, we have different splits and in each split, we will have list of sentences and labels.

**Note:** This task publishes a 'raw dataset' as specified in `self.save(dataset_splits, "raw_dataset")`. Only the items saved in such a manner can be passed to the downstream tasks.

Here is the complete implementation of DatasetFetchTask:


```python
from datasets import load_dataset


class DatasetFetchTask(Task):
    def __init__(self):
        super().__init__()

    @staticmethod
    def _get_split(dataset, split):
        if split == "test":
            return dataset[split]
        elif split in ["train", "val"]:
            splitted = list(dataset["train"])
            split_index = int(0.7 * len(splitted))
            return splitted[:split_index] if split == "train" else splitted[split_index:]

    @staticmethod
    def _get_sentences_and_labels(dataset) -> Tuple[List[str], List[str]]:
        sentences = []
        labels = []
        for item in dataset:
            sentences.append(item["text"])
            label = item["coarse_label"] if "coarse_label" in item else item["label-coarse"]
            labels.append(label)
        return sentences, labels

    def run(self):
        dataset = load_dataset("trec")
        splits = ["train", "val", "test"]
        dataset_splits = {}
        for split in splits:
            dataset_split = DatasetFetchTask._get_split(dataset, split)
            sentences, labels = DatasetFetchTask._get_sentences_and_labels(dataset_split)
            split_results = {"sentences": sentences, "labels": labels}
            dataset_splits[split] = split_results
        self.save(dataset_splits, "raw_dataset")

```

### **2. Dataset Pre-processing:**
Now that we have our raw datasets prepared, next, we can apply some pre-processing to clean them up a bit, such as *removing punctuations*, *removing digits* and *making lower case* etc. 
We will implement this logic into a PreProcessTask. Additionally, this task takes a list of pre-processing steps as parameters.

**Note**, this task assumes that the raw dataset is available to it as arguments of run() method. This will be automatically passed by FluidML. 
Later, when we create the PreProcess Task, we just have to connect the DatasetFetchTask as a predecessor. More on this later, when we model the flow/pipeline.

Therefore, PreProcessTask just has to implement its own logic and save the pre-processed sentences. In this way, we can implement the tasks individually (**separation of concerns**)

Here is the complete implementation of PreProcessTask:

```python
import string
import re


class PreProcessTask(Task):
    def __init__(self, pre_processing_steps: List[str]):
        super().__init__()
        self._pre_processing_steps = pre_processing_steps

    def _pre_process(self, text: Dict) -> str:
        pre_processed_text = text
        for step in self._pre_processing_steps:
            if step == "lower_case":
                pre_processed_text = pre_processed_text.lower()
            if step == "remove_punct":
                pre_processed_text = pre_processed_text.translate(str.maketrans("", "", string.punctuation))
            if step == "remove_digits":
                pre_processed_text = re.sub(r"\d+", "<num>", pre_processed_text)
        return pre_processed_text

    def run(self, raw_dataset: Dict):
        pre_processed_splits = {}
        for split in ["train", "val", "test"]:
            pre_processed_sentences = [self._pre_process(sentence) for sentence in raw_dataset[split]["sentences"]]
            pre_processed_splits[split] = {"sentences": pre_processed_sentences}
        self.save(pre_processed_splits, "pre_processed_dataset")
```

### **3. Featurization:**

Our datasets are prepared and sentences are pre-processed, so now we can convert these to numerical vectors which can be then fed to classifiers. To this end, we would like to implement two featurizers, namely a TFIDF featurizer and a glove featurizer. They can be implemented as two independent tasks which take preprocessed sentences and save vectorized sentences.

You may have guessed the pattern already, this task expects pre_processed dataset, which was produced by PreProcessTask. 

```python
from sklearn.feature_extraction.text import TfidfVectorizer
from flair.data import Sentence
from flair.embeddings import WordEmbeddings, DocumentPoolEmbeddings

class TFIDFFeaturizeTask(Task):
    def __init__(self, min_df: int, max_features: int):
        super().__init__()
        self._min_df = min_df
        self._max_features = max_features

    def run(self, pre_processed_dataset: Dict):
        tfidf_model = TfidfVectorizer(min_df=self._min_df, max_features=self._max_features)
        tfidf_model.fit(pre_processed_dataset["train"]["sentences"])
        featurized_splits = {}
        for split in ["train", "val", "test"]:
            tfidf_vectors = tfidf_model.transform(pre_processed_dataset[split]["sentences"]).toarray()
            featurized_splits[split] = {"vectors": tfidf_vectors}
        self.save(featurized_splits, "tfidf_featurized_dataset")


class GloveFeaturizeTask(Task):
    def __init__(self):
        super().__init__()

    def run(self, pre_processed_dataset: Dict):
        featurized_splits = {}
        for split in ["train", "val", "test"]:
            sentences = [Sentence(sent) for sent in pre_processed_dataset[split]["sentences"]]
            embedder = DocumentPoolEmbeddings([WordEmbeddings("glove")])
            embedder.embed(sentences)
            glove_vectors = [sent.embedding.cpu().numpy() for sent in sentences]
            glove_vectors = np.array(glove_vectors).reshape(len(glove_vectors), -1)
            featurized_splits[split] = {"vectors": glove_vectors}
        self.save(featurized_splits, "glove_featurized_dataset")
```

### **4. Training a classifier**

We are all set to train a simple classifier. For this tutorial, let's stick with a simple logistic regression model from Sklearn.
For the inputs, we can stack the glove and tfidf vectors (obtained from featurization task results) and for the targets, we can just use the labels from the raw dataset. In the end, this task returns a trained SKlearn classifier.

**Note:** You are not limited just to Sklearn. You can train any kind of model using your favorite library (PyTorch, TensorFlow, Keras, PyTorch Lightning, etc) 

```python

from sklearn.linear_model import LogisticRegression

class TrainTask(Task):
    def __init__(self, max_iter: int, balanced: str):
        super().__init__()
        self._max_iter = max_iter
        self._class_weight = "balanced" if balanced else None

    def run(self, raw_dataset: Dict, tfidf_featurized_dataset: Dict, glove_featurized_dataset: Dict):
        model = LogisticRegression(max_iter=self._max_iter, class_weight=self._class_weight)
        stacked_vectors = np.hstack(
            (tfidf_featurized_dataset["train"]["vectors"], glove_featurized_dataset["train"]["vectors"])
        )
        model.fit(stacked_vectors, raw_dataset["train"]["labels"])
        self.save(model, "trained_model")
```

### **5. Evaluation of classifier**

Now, that we have trained a classifier, it is time to evaluate this classifier on all of the dataset splits. This task is straightforward, we will get the featurized dataset splits and the trained model from the results. 
At the end, EvaluateTask saves a nested dictionary containing classification reports for each of train, val and test splits.


```python
from sklearn.metrics import classification_report

class EvaluateTask(Task):
    def __init__(self):
        super().__init__()

    def run(
        self,
        raw_dataset: Dict,
        tfidf_featurized_dataset: Dict,
        glove_featurized_dataset: Dict,
        trained_model: LogisticRegression,
    ):
        evaluation_results = {}
        for split in ["train", "val", "test"]:
            stacked_vectors = np.hstack(
                (tfidf_featurized_dataset[split]["vectors"], glove_featurized_dataset[split]["vectors"])
            )
            predictions = trained_model.predict(stacked_vectors)
            report = classification_report(raw_dataset[split]["labels"], predictions, output_dict=True)
            evaluation_results[split] = {"classification_report": report}
        self.save(evaluation_results, "evaluation_results")
```

## **Create and Run the Pipeline/Task-Graph via FluidML**

So far, we have looked into implementing our individual pipeline steps using FluidML's Task class and it was very straightforward.
You might be wondering, how to put these tasks together and make them work together as a single pipeline?

Thanks to FluidML's TaskSpec API, you can connect these tasks like Lego blocks :)

### **1. Instantiate Task Specifications**
TaskSpec is a simple wrapper that allows to specify task details task arguments which will be used during instantiation of the task.
Let's go ahead and create specs for all our tasks.

**Note 2:** We need to pass the `reduce=True` parameter to the ModelSelection task, so that it will get access to *all* previous evaluation results. So, it *reduces* the task graph.

In [3]:
# create all task specs
dataset_fetch_task = TaskSpec(task=DatasetFetchTask)
pre_process_task = TaskSpec(task=PreProcessTask, config={"pre_processing_steps": ["lower_case", "remove_punct"]})
featurize_task_1 = TaskSpec(task=GloveFeaturizeTask)
featurize_task_2 = TaskSpec(task=TFIDFFeaturizeTask, config={"min_df": 5, "max_features": 2000})
train_task = TaskSpec(task=TrainTask, config={"max_iter": 50, "balanced": True})
evaluate_task = TaskSpec(task=EvaluateTask)

### **2. Registering Task Dependencies**
More importantly, TaskSpec also provides `requires()` method to specify predecessor tasks which need to be executed before that particular task.

For instance, in our example, we would need DatasetFetchTask to be finished before we start to run PreProcessTask. Similarly,
PreProcessTask is required for both FeaturizeTasks.

Using these task dependencies, FluidML creates a task graph and schedules the tasks considering the dependencies.
Not just that, it automatically collects the results from predecessor tasks and makes it available to `run()` method.


In [4]:
# dependencies between tasks
pre_process_task.requires(dataset_fetch_task)
featurize_task_1.requires(pre_process_task)
featurize_task_2.requires(pre_process_task)
train_task.requires([dataset_fetch_task, featurize_task_1, featurize_task_2])
evaluate_task.requires([dataset_fetch_task, featurize_task_1, featurize_task_2, train_task])

### **3. Creating a Final list of Task Specs**
We can just hold all these tasks in a list which we will pass it to FluidML.

In [5]:
# all task specs
tasks = [
    dataset_fetch_task,
    pre_process_task,
    featurize_task_1,
    featurize_task_2,
    train_task,
    evaluate_task,
]

### **4. Run the Pipeline/Task-Graph using Swarm & Flow**

All that is left is to create the `Flow` object, which will handle executing the created task graph in parallel and builds the user defined task specifier graph and the expanded task graph. The latter two can be easily accessed as attributes and visualized using our console rendering utility function. Finally, we execute the expanded task graph by calling `flow.run()`


In [6]:
# create flow
flow = Flow(tasks=tasks)
# run the pipeline
results = flow.run(project_name="sklearn_text_classification_example")

[03/06/23 21:55:22] INFO     MainProcess  Execute run "nautical-raise" using multiprocessing 
                             with 2 workers

[03/06/23 21:55:25] INFO     Dolphin-2    Started task "DatasetFetchTask"

[03/06/23 21:55:26] INFO     Dolphin-2    Finished task "DatasetFetchTask" [1/6 - 17%]

[03/06/23 21:55:26] INFO     Dolphin-1    Started task "PreProcessTask"

[03/06/23 21:55:26] INFO     Dolphin-1    Finished task "PreProcessTask" [2/6 - 33%]

...

[03/06/23 21:56:17] INFO     Dolphin-1    Started task "TrainTask"

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


[03/06/23 21:56:18] INFO     Dolphin-1    Finished task "TrainTask" [5/6 - 83%]

[03/06/23 21:56:19] INFO     Dolphin-2    Started task "EvaluateTask"

[03/06/23 21:56:19] INFO     Dolphin-2    Finished task "EvaluateTask" [6/6 - 100%]

### **5. Results**
We can now go over the results and fetch a task's result using its name, which would give task results and task_config (up until that task in the graph)

In [7]:
print(results["EvaluateTask"]["config"])

{'DatasetFetchTask': {}, 'PreProcessTask': {'pre_processing_steps': ['lower_case', 
'remove_punct']}, 'GloveFeaturizeTask': {}, 'TFIDFFeaturizeTask': {'min_df': 5, 
'max_features': 2000}, 'TrainTask': {'max_iter': 50, 'balanced': True}, 'EvaluateTask': {}}

In [8]:
print(results["EvaluateTask"]["results"]["evaluation_results"]["test"])

{
    'classification_report': {
        '0': {
            'precision': 0.8592592592592593,
            'recall': 0.8405797101449275,
            'f1-score': 0.8498168498168497,
            'support': 138
        },
        '1': {
            'precision': 0.7945205479452054,
            'recall': 0.6170212765957447,
            'f1-score': 0.6946107784431137,
            'support': 94
        },
        '2': {
            'precision': 0.35,
            'recall': 0.7777777777777778,
            'f1-score': 0.48275862068965514,
            'support': 9
        },
        '3': {
            'precision': 0.8955223880597015,
            'recall': 0.9230769230769231,
            'f1-score': 0.9090909090909091,
            'support': 65
        },
        '4': {
            'precision': 0.9433962264150944,
            'recall': 0.8849557522123894,
            'f1-score': 0.91324200913242,
            'support': 113
        },
        '5': {
            'precision': 0.7878787878787878,
            'recall': 0.9629629629629629,
            'f1-score': 0.8666666666666665,
            'support': 81
        },
        'accuracy': 0.838,
        'macro avg': {
            'precision': 0.7717628682596748,
            'recall': 0.8343957337951209,
            'f1-score': 0.7860309723066025,
            'support': 500
        },
        'weighted avg': {
            'precision': 0.8500872398231903,
            'recall': 0.838,
            'f1-score': 0.8388004443149147,
            'support': 500
        }
    }
}

## **Grid Search:**

We can extend this pipeline and include grid search to find hyper-parameter tuning on the whole pipeline.
To enable grid search on a particular task, we just have specify `expand` parameter with either `"product"` or `"zip"`.

Depending on the selected grid search expansion method, Flow would expand this task into several task instances with provided combinations of `max_iter` and `balanced`. Namely, `expand="product"` (default) will create 4 task instances for each explicit cross-product parameter combination. Alternatively, `expand="zip"` expands the config parameters by zipping over the different parameter lists, creating 2 task instances for the above example with the combinations `[(50, True), (100, False)]`. When choosing `zip` one has to make sure that all parameter lists are of equal lengths. 

Any successor tasks (for instance, evaluate task) in the task graph will also be automatically extended. In our example, we have 8 evaluate tasks.

### **Model Selection Task:**

Ok that's nice. We have now several evaluation tasks and once we run this task graph through Flow, we will have several evaluation tasks and their results.
We can implement a model selection task, which consolidates these results and fetches the best config for the pipeline.

```python
class ModelSelectionTask(Task):
    def __init__(self):
        super().__init__()

    def run(self, evaluation_results: List[Sweep]):
        sorted_results = sorted(
            evaluation_results,
            key=lambda model_result: model_result.value["val"]["classification_report"]["macro avg"]["f1-score"],
            reverse=True,
        )
        self.save(sorted_results[0].config, "best_config")
        self.save(sorted_results[0].value, "best_performance")
```


**Note:** Now, we can attach the EvaluationTask as a predecessor to ModelSelectionTask. However, if we do it naively, we would end up with 4 model selection tasks since we have 4 evaluation tasks.
So, we just have to specify `reduce=True` so only one instance of model selection task is created and all of the evaluation tasks are attached as parents to it.

```python
model_selection_task = TaskSpec(task=ModelSelectionTask, reduce=True)
model_selection_task.requires(evaluate_task)
```
Finally, putting all of these together:

In [9]:
# create all task specs
dataset_fetch_task = TaskSpec(task=DatasetFetchTask)
pre_process_task = TaskSpec(task=PreProcessTask, config={"pre_processing_steps": ["lower_case", "remove_punct"]})
featurize_task_1 = TaskSpec(task=GloveFeaturizeTask)
featurize_task_2 = TaskSpec(
    task=TFIDFFeaturizeTask, config={"min_df": 5, "max_features": [1000, 2000]}, expand="product"
)
train_task = TaskSpec(task=TrainTask, config={"max_iter": [50, 100], "balanced": [True, False]}, expand="product")
evaluate_task = TaskSpec(task=EvaluateTask)
model_selection_task = TaskSpec(task=ModelSelectionTask, reduce=True)

In [10]:
# dependencies between tasks
pre_process_task.requires(dataset_fetch_task)
featurize_task_1.requires(pre_process_task)
featurize_task_2.requires(pre_process_task)
train_task.requires(dataset_fetch_task, featurize_task_1, featurize_task_2)
evaluate_task.requires(dataset_fetch_task, featurize_task_1, featurize_task_2, train_task)
model_selection_task.requires(evaluate_task)

In [11]:
# all tasks
tasks = [dataset_fetch_task,
         pre_process_task,
         featurize_task_1, featurize_task_2,
         train_task,
         evaluate_task,
         model_selection_task]

flow = Flow(tasks=tasks)

# run the pipeline
results = flow.run(project_name="sklearn_text_classification_example")

[03/06/23 21:56:48] INFO     MainProcess  Execute run "discrete-pizza" using multiprocessing 
                             with 8 workers

[03/06/23 21:56:50] INFO     Dolphin-4    Started task "DatasetFetchTask"

[03/06/23 21:56:51] INFO     Dolphin-4    Finished task "DatasetFetchTask" [1/22 - 5%]

[03/06/23 21:56:51] INFO     Dolphin-6    Started task "PreProcessTask"

[03/06/23 21:56:51] INFO     Dolphin-6    Finished task "PreProcessTask" [2/22 - 9%]

...

[03/06/23 21:58:31] INFO     Dolphin-3    Finished task "EvaluateTask-3" [19/22 - 86%]

[03/06/23 21:58:31] INFO     Dolphin-7    Finished task "EvaluateTask-7" [20/22 - 91%]

[03/06/23 21:58:31] INFO     Dolphin-1    Finished task "EvaluateTask-8" [21/22 - 95%]

[03/06/23 21:58:31] INFO     Dolphin-5    Started task "ModelSelectionTask"

[03/06/23 21:58:31] INFO     Dolphin-5    Finished task "ModelSelectionTask" [22/22 - 100%]

### **Choosing the best Config and best performance**

In [12]:
print(results["ModelSelectionTask"][0].results["best_config"])
print(results["ModelSelectionTask"][0].results["best_performance"]["test"])

{'DatasetFetchTask': {}, 'PreProcessTask': {'pre_processing_steps': ['lower_case', 
'remove_punct']}, 'GloveFeaturizeTask': {}, 'TFIDFFeaturizeTask': {'min_df': 5, 
'max_features': 1000}, 'TrainTask': {'max_iter': 100, 'balanced': False}, 'EvaluateTask': {}}

{
    'classification_report': {
        '0': {
            'precision': 0.8552631578947368,
            'recall': 0.9420289855072463,
            'f1-score': 0.8965517241379309,
            'support': 138
        },
        '1': {
            'precision': 0.8292682926829268,
            'recall': 0.723404255319149,
            'f1-score': 0.7727272727272727,
            'support': 94
        },
        '2': {
            'precision': 1.0,
            'recall': 0.7777777777777778,
            'f1-score': 0.8750000000000001,
            'support': 9
        },
        '3': {
            'precision': 0.9104477611940298,
            'recall': 0.9384615384615385,
            'f1-score': 0.9242424242424243,
            'support': 65
        },
        '4': {
            'precision': 0.9696969696969697,
            'recall': 0.8495575221238938,
            'f1-score': 0.9056603773584906,
            'support': 113
        },
        '5': {
            'precision': 0.8172043010752689,
            'recall': 0.9382716049382716,
            'f1-score': 0.8735632183908048,
            'support': 81
        },
        'accuracy': 0.876,
        'macro avg': {
            'precision': 0.8969800804239888,
            'recall': 0.8615836140213129,
            'f1-score': 0.8746241694761538,
            'support': 500
        },
        'weighted avg': {
            'precision': 0.8798518914842701,
            'recall': 0.876,
            'f1-score': 0.8748190049486405,
            'support': 500
        }
    }
}